<a href="https://colab.research.google.com/github/yukinaga/automl/blob/main/section_2/02_automl_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AutoMLによる「クラスタリング」
ライブラリPyCaretを使い、AutoMLによる「クラスタリング」の機械学習を実装します。  
様々なタンパク質の測定値から、マウスの分類を行います。  

## PyCaretのインストール
AutoMLをサポートするライブラリ、PyCaretをバージョンを指定してインストールします。

In [ ]:
!pip install numpy==1.21.4 numba==0.53
!pip install pycaret==2.3.6
!pip install pandas-profiling==3.1.0

## Google Colabの設定
Google Colab環境でPyCaretのインタラクティブな要素を表示するためには、以下のコードを実行する必要があります。

In [ ]:
from pycaret.utils import enable_colab

enable_colab()

## データセットの読み込み

今回は、UCIのMice Protein Expressionというデータセットを使います。  
このデータセットは、大脳皮質の核の部分で検出可能なシグナルを出した、77のタンパク質／タンパク質修飾の発現レベルで構成されています。  
このデータセットには、タンパク質ごとに合計1080の測定値が含まれおり、各測定値は独立したマウスのものです。  

https://archive.ics.uci.edu/ml/datasets/Mice+Protein+Expression

In [ ]:
from pycaret.datasets import get_data

dataset = get_data("mice")  # データの取得
dataset.shape  # データの形状

訓練済みモデルの検証に用いる「未知のデータ」を訓練データから取り出します。  

In [ ]:
data = dataset.sample(frac=0.9, random_state=786)  # 訓練データ
data_unseen = dataset.drop(data.index)  # 未知のデータ

data.reset_index(drop=True, inplace=True)  # インデックスを初期化
data_unseen.reset_index(drop=True, inplace=True)  # インデックスを初期化

print("訓練データの形状: " + str(data.shape))
print("未知のデータの形状: " + str(data_unseen.shape))

## 環境の設定
PyCaretの環境を設定します。  
setup関数はPyCaretの環境を初期化しますが、PyCaretの他の関数を実行する前に呼び出す必要があります。  

以下のコードを実行すると、setup関数が実行されてすべての特徴のデータ型が自動的に推定されます。  
この際に、必ずしも正しく推論されるとは限らないことにご注意ください。     
データの型に問題が無ければ、空白を入力することで設定を完了することができます。  

In [ ]:
from pycaret.clustering import setup

clf = setup(data=data, normalize = True, ignore_features = ['MouseID'], session_id=123)  # 環境の初期化

## モデルの作成
models関数により、全ての使用可能な機械学習モデルを確認することができます。

In [ ]:
from pycaret.clustering import models

models()  # 機械学習モデルの一覧

create_model関数は、「交差検証」を用いて個別のモデルの訓練と評価を行います。  
デフォルトでは、データを10に分割し10通りのテストデータ/訓練データの組み合わせを試します。     
  
今回は、上記の中で最も有名な「k平均法」のモデルを作成します。  
`num_clusters`でクラスタの数（グループの数）を指定することができます。

In [ ]:
from pycaret.clustering import create_model

kmeans = create_model("kmeans", num_clusters=4)  # 線形回帰のモデルを作成

訓練済みモデルの概要を表示します。  

In [ ]:
print(kmeans)

## ラベルの割当て
`assign_model`関数により、各サンプルに所属するクラスタを表すラベルを割り当てることができます。  

In [ ]:
from pycaret.clustering import assign_model

kmean_assigned = assign_model(kmeans)
kmean_assigned.head()

## モデルを評価する
plot_model関数により、様々な角度からモデルを分析することができます。  
https://pycaret.org/plot-model/

plot_model関数でplotを指定しない場合、主成分分析（PCA）の結果が表示されます。  
この場合、2つの軸に各グループの分布を圧縮することになります。  

In [ ]:
from pycaret.clustering import plot_model

plot_model(kmeans)

エルボー法（Elbow method）は、適切なクラスタ数を見つけるための手法です。

In [ ]:
plot_model(kmeans, plot="elbow")  # エルボー法

この例は、最適なクラスタ数を点線で示唆しています。

シルエット図（Silhouette Plot）は、クラスタ内のサンプルがどの程度密にグループされているか、隣接するクラスターとどれくらい離れているかを可視化します。  


In [ ]:
plot_model(kmeans, plot="silhouette")  # シルエット図

分布図（Distribution）は、各クラスタの大きさを示します。  



In [ ]:
plot_model(kmeans, plot="distribution")  # クラスタのサイズを確認

## モデルの保存と読み込み

`save_model`関数によりモデルを保存することができます。  

In [ ]:
from pycaret.clustering import save_model

save_model(kmeans, "kmeans_model")

`load_model`関数により、保存されたモデルを読み込むことができます。

In [ ]:
from pycaret.clustering import load_model

loaded_kmeans = load_model("kmeans_model")

## 未知のデータで予測する
「未知のデータ」が所属すべきクラスタを予測します。  

In [ ]:
from pycaret.clustering import predict_model

unseen_predictions = predict_model(loaded_kmeans, data=data_unseen)
unseen_predictions.head()